In [1]:
import torch
import torchsummary
import os
import sys
import pytorch_lightning as pl
import torch.optim as optim
import torch.nn as nn

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(ROOT_DIR)

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(ROOT_DIR)

from utils import helper, input_getter, qai_hub_jobs, tfhelper
import training
import rcvit_extended_train
import rcvit

/home/centar15-desktop1/LPCV_2025_T1/.venv310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-22 09:33:38.631232: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-22 09:33:38.639502: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742632418.648265  309473 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742632418.650897  309473 cud

In [2]:
model: nn.Module = rcvit_extended_train.get_rcvit_extended(pretrained_path = "CASVIT_t.pth")
model.dist = False
# model = rcvit.rcvit_t()

/home/centar15-desktop1/LPCV_2025_T1/src/casvit/rcvit_extended_train.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained = torch.load(pretrained_path)


Re-initializing classifier head
Loaded 20.8M/31.1M parameters


In [9]:
# import torch

import training.lightning_model
import training.lightning_train_function

weights = torch.randn([64])

# lightningModel = training.lightning_model.LightningModel(modelBase, class_weights=weights)
# lightningModel.load_from_checkpoint("../../models/CASVIT_T_SD_BALANCING/Ver1.0/CASVIT_T_SD_BALANCING-epoch=14-train_loss=0.0492-val_loss=0.1135.ckpt")

lightningModel = training.lightning_model.LightningModel.load_from_checkpoint("../../models/CASVIT_T_WITH_512_FINAL_LAYER/Ver1.1/CASVIT_T_WITH_512_FINAL_LAYER-epoch=22-train_loss=0.3280-val_loss=0.5063.ckpt", model = model)

modelFp32 = lightningModel.model
modelFp32.eval()

/home/centar15-desktop1/LPCV_2025_T1/src/training/lightning_model.py:18: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/centar15-desktop1/LPCV_2025_T1/src/training/lightning_model.py:23: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Init device
cpu


RCViT(
  (patch_embed): Sequential(
    (0): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
  )
  (network): ModuleList(
    (0): Sequential(
      (0): AdditiveBlock(
        (local_perception): LocalIntegration(
          (network): Sequential(
            (0): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1))
            (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=96)
            (3): GELU(approximate='none')
            (4): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1))
          )
        )
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, aff

In [10]:
modelFp32(torch.randn((1, 3, 224, 224)))

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [6]:
modelFp32.eval()
modelFp32.cuda()


torchsummary.summary(modelFp32.cuda(), (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 48, 112, 112]           1,344
       BatchNorm2d-2         [-1, 48, 112, 112]              96
              ReLU-3         [-1, 48, 112, 112]               0
            Conv2d-4           [-1, 96, 56, 56]          41,568
       BatchNorm2d-5           [-1, 96, 56, 56]             192
              ReLU-6           [-1, 96, 56, 56]               0
            Conv2d-7           [-1, 96, 56, 56]           9,312
       BatchNorm2d-8           [-1, 96, 56, 56]             192
            Conv2d-9           [-1, 96, 56, 56]             960
             GELU-10           [-1, 96, 56, 56]               0
           Conv2d-11           [-1, 96, 56, 56]           9,312
 LocalIntegration-12           [-1, 96, 56, 56]               0
      BatchNorm2d-13           [-1, 96, 56, 56]             192
           Conv2d-14          [-1, 288,

In [11]:
input = input_getter.local_image_getter("/home/centar15-desktop1/LPCV_2025_T1/datasets/imagenet/train/banana/0000316013.jpeg")
input_tensor = input.get_input_torch()

In [12]:
modelFp32(input.get_input_torch().cuda())

tensor([[-2.1998e-01, -1.4891e+00, -3.6274e-01, -2.6878e-02, -5.1499e-01,
         -5.9988e-01, -1.1006e+00, -1.1575e+00,  8.0365e-01,  2.1609e-01,
          7.6623e-01, -3.3292e-01, -5.7637e-01, -4.9860e-01, -2.2946e+00,
         -1.7250e-01,  2.1389e-01,  3.1646e-01, -1.9605e-01, -4.6308e-01,
         -1.5258e-01,  1.7325e-01, -1.9079e-01, -2.0852e-01,  3.8172e-01,
          4.3666e-01, -4.4710e-01,  5.5896e-01,  2.9313e-01, -4.2712e-01,
         -3.3304e-03, -9.6425e-01, -1.2692e-01, -2.6702e-01, -1.7247e-01,
          6.6138e+00, -4.8179e-01, -1.6574e-01, -5.4325e-02,  1.2667e-01,
         -4.4822e-01,  9.3577e-02, -7.3777e-01, -3.2085e-01, -2.4339e-01,
          1.0207e-02, -6.3629e-01,  9.7662e-03, -7.2313e-01, -4.3015e-01,
          6.4800e-01,  6.4997e-01,  4.2009e-01,  6.2996e-01,  6.1407e-01,
         -1.1645e-01,  6.3233e-01, -1.0669e-01,  5.0293e-02,  2.0683e-01,
         -2.7261e-01,  3.2725e-02, -4.0474e-02,  1.0070e+00]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [13]:
from dataset import DatasetReader
from dataset.utils import GLOBAL_CLASSES
from torchvision import transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    # transforms.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
    transforms.Resize((224, 224)),
    # transforms.ToTensor(),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])


In [14]:
# categories = helper.get_imagenet_categories()
categories = GLOBAL_CLASSES

helper.print_probablities_from_output(modelFp32.cpu()(transform(input.get_input_torch())), categories=categories, modelname = 'Torch Local')

Top-5 predictions for Torch Local on :
35 Banana                92.5%
63 Hair Drier             0.4%
8 Traffic Light          0.3%
10 Parking Meter          0.2%
51 Remote                 0.2%


In [15]:
idx = 0
from PIL import Image

In [569]:
import matplotlib.pyplot as plt
import skimage as ski
from utils.helper import print_probablities_from_output
categories = GLOBAL_CLASSES


folderpath = "/home/centar15-desktop1/Desktop/SliciceZaTest/Igrica"

# folderpath = "/home/centar15-desktop1/Desktop/imagenet_test/"
categories = GLOBAL_CLASSES


transformBest = transforms.Compose([
    transforms.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
    # transforms.RandomEqualize(p = 1),
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])

import kornia.enhance as ke

class KorniaHistogramEqualization:
    def __call__(self, img:torch.Tensor) -> torch.Tensor:
        return ke.equalize(img)


transformHistEq = transforms.Compose([
    transforms.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
    # transforms.RandomEqualize(p = 1),
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    KorniaHistogramEqualization(),
    
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.288, 0.288, 0.288])  
])


image = Image.open(folderpath + "/image_" + f"{idx}.jpg")
# image = Image.open(folderpath + "/imagenet_test_" + f"{idx}.JPEG")

# image = Image.open(folderpath + "ILSVRC2012_test_" + str(idx).zfill(8) + ".JPEG")

if image.mode == 'RGBA':
    # Convert the image to RGB (remove alpha channel)
    image = image.convert('RGB')

plt.imshow(image)
plt.show()

modelFp32.cpu()
modelFp32.eval()
# modelDesktop.cpu()
# modelDesktop.eval()

print_probablities_from_output(modelFp32(transformBest(image).unsqueeze(0)), categories, 5, 'Model trained on jet')
print("")
# print_probablities_from_output(modelDesktop(transformBest(image).unsqueeze(0)), categories, 5, 'Model trained on desktop')
# print("")
# print_probablities_from_output(modelNoHistEq(transformBest(image).unsqueeze(0).cuda()), categories, 5, 'No EQ Stable diffusion training')

idx = idx + 1

FileNotFoundError: [Errno 2] No such file or directory: '/home/centar15-desktop1/Desktop/SliciceZaTest/Igrica/image_533.jpg'

In [6]:
import qai_hub

input_shape = (1, 3, 224, 224)
dummy_input = torch.randn(input_shape)

pt_model = torch.jit.trace(model, dummy_input)

compile_job = qai_hub.submit_compile_job(
    pt_model,
    name="CASVIT_T_MODIFIED", # Replace with your model name
    device=qai_hub.Device("Snapdragon 8 Elite QRD"),
    input_specs=dict(image=input_shape)
)

# compile_job.modify_sharing(add_emails=['lowpowervision@gmail.com']) ## Share your model for submission

Uploading tmps1pn1c03.pt


100%|██████████| 120M/120M [00:05<00:00, 21.4MB/s] 


Scheduled compile job (j5wwk4qm5) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/j5wwk4qm5/



In [7]:
qai_hub.submit_profile_job(compile_job.get_target_model(), device=qai_hub.Device("Snapdragon 8 Elite QRD"), name = "CASVIT_T_MODIFIED")

Waiting for compile job (j5wwk4qm5) completion. Type Ctrl+C to stop waiting at any time.
    ✅ SUCCESS                          
Scheduled profile job (jgn04krj5) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jgn04krj5/



ProfileJob
----------
job_id  : jgn04krj5
url     : https://app.aihub.qualcomm.com/jobs/jgn04krj5/
status  : CREATED
model   : Model(model_id='mnoeo837q', name='job_j5wwk4qm5_optimized_tflite')
name    : CASVIT_T_MODIFIED
options : 
shapes  : {}
device  : Device(name='Snapdragon 8 Elite QRD', os='15', attributes=['os:android', 'framework:tflite', 'framework:onnx', 'vendor:qualcomm', 'abi:aarch64-android', 'format:phone', 'chipset:qualcomm-snapdragon-8-elite', 'chipset:sm8750', 'hexagon:v79', 'soc-model:69', 'htp-supports-fp16:true', 'framework:qnn'])
date    : 2025-03-21 20:23:27

In [577]:
from aimet_torch.cross_layer_equalization import equalize_model

from aimet_torch.quantsim import QuantizationSimModel
from aimet_common.defs import QuantScheme
from torchvision import transforms
from torch.utils.data import DataLoader

# Dummy input to define the model input size
dummy_input = torch.rand(1, 3, 224, 224)

# Step 1: Create QuantizationSimModel
sim = QuantizationSimModel(model, dummy_input=dummy_input,
                                     quant_scheme=QuantScheme.post_training_tf_enhanced,
                                     default_param_bw=8, default_output_bw=8)

from dataset import DatasetReader

# Step 2: Compute Encodings (calibration)
def calibration_function(model, eval_iterations = 100, use_cuda = False):
    for i in range(eval_iterations):
        data = torch.randn(1, 3, 224, 224)
        print(i)
        model(data)


2025-03-21 21:12:23,076 - root - INFO - AIMET


/home/centar15-desktop1/LPCV_2025_T1/.venv310/lib/python3.10/site-packages/onnxscript/converter.py:823: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/usr/lib/python3.10/abc.py:106: FutureWarning: `NLLLoss2d` has been deprecated. Please use `NLLLoss` instead as a drop-in replacement and see https://pytorch.org/docs/main/nn.html#torch.nn.NLLLoss for more details.
  cls = super().__new__(mcls, name, bases, namespace, **kwargs)


2025-03-21 21:12:25,868 - ConnectedGraph - WARNING - Unable to isolate model outputs.
2025-03-21 21:12:26,085 - Quant - INFO - No config file provided, defaulting to config file at /home/centar15-desktop1/LPCV_2025_T1/.venv310/lib/python3.10/site-packages/aimet_common/quantsim_config/default_config.json
2025-03-21 21:12:26,097 - Quant - INFO - Unsupported op type Squeeze
2025-03-21 21:12:26,097 - Quant - INFO - Unsupported op type Mean
2025-03-21 21:12:26,128 - Quant - INFO - Selecting DefaultOpInstanceConfigGenerator to compute the specialized config. hw_version:default


In [578]:
print(sim.model)

RCViT(
  (patch_embed): Sequential(
    (0): QuantizedConv2d(
      3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)
      (param_quantizers): ModuleDict(
        (weight): QuantizeDequantize(shape=(), qmin=-128, qmax=127, symmetric=True)
        (bias): None
      )
      (input_quantizers): ModuleList(
        (0): QuantizeDequantize(shape=(), qmin=0, qmax=255, symmetric=False)
      )
      (output_quantizers): ModuleList(
        (0): None
      )
    )
    (1): QuantizedBatchNorm2d(
      48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (param_quantizers): ModuleDict(
        (weight): None
        (bias): None
      )
      (input_quantizers): ModuleList(
        (0): None
      )
      (output_quantizers): ModuleList(
        (0): None
      )
    )
    (2): QuantizedReLU(
      (param_quantizers): ModuleDict()
      (input_quantizers): ModuleList(
        (0): None
      )
      (output_quantizers): ModuleList(
        (0): QuantizeDequantize(sha

In [584]:
sim.model(dummy_input)

DequantizedTensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                    0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                    0., 0., 0., 0.]], grad_fn=<AliasBackward0>)

In [580]:
sim.compute_encodings(calibration_function)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [583]:
helper.print_probablities_from_output(sim.model(transform(input.get_input_torch())), categories=categories, modelname = 'Torch Local')

Top-5 predictions for Torch Local on :
63 Hair Drier             1.6%
62 Teddy Bear             1.6%
61 Vase                   1.6%
60 Clock                  1.6%
59 Book                   1.6%


In [11]:
import os

folderpath = "q1"

os.makedirs(folderpath, exist_ok=True)

sim.export(path=folderpath, filename_prefix='CASVIT_T_EXTENDED_BIG', dummy_input=dummy_input)


2025-03-21 20:28:03,493 - Utils - INFO - successfully created onnx model with 645/1198 node names updated
2025-03-21 20:28:03,718 - Quant - WARNING - number of input quantizers: 1 available for layer: network.0.0.norm1 doesn't match with number of input tensors: 3
2025-03-21 20:28:03,723 - Quant - WARNING - number of input quantizers: 1 available for layer: network.0.0.norm2 doesn't match with number of input tensors: 3
2025-03-21 20:28:03,726 - Quant - WARNING - number of input quantizers: 1 available for layer: network.0.1.norm1 doesn't match with number of input tensors: 3
2025-03-21 20:28:03,731 - Quant - WARNING - number of input quantizers: 1 available for layer: network.0.1.norm2 doesn't match with number of input tensors: 3
2025-03-21 20:28:03,733 - Quant - WARNING - number of input quantizers: 1 available for layer: network.0.2.norm1 doesn't match with number of input tensors: 3
2025-03-21 20:28:03,738 - Quant - WARNING - number of input quantizers: 1 available for layer: netw

In [12]:
import qai_hub

name = 'CASVIT_T_EXTENDED'

compile_job = qai_hub.submit_compile_job(
    model="Q2.aimet",
    device=qai_hub.Device("Snapdragon 8 Elite QRD"),
    name = name + 'tflite'
)
assert isinstance(compile_job, qai_hub.CompileJob)

# Compile to a QNN Model Library
compile_job_qnn = qai_hub.submit_compile_job(
    model="Q2.aimet",
    device=qai_hub.Device("Snapdragon 8 Elite QRD"),
    options="--target_runtime qnn_lib_aarch64_android --quantize_full_type int8",
    name = name + 'qnn'
)
assert isinstance(compile_job_qnn, qai_hub.CompileJob)

Uploading Q2.aimet.zip
2025-03-21 20:29:30,283 - root - INFO - Uploading asset to https://tetrahub-qprod-userdata.s3-accelerate.amazonaws.com/models/mnjlo5g9q_TdIqELt6pRSXxhFg.aimet.zip?uploadId=GvlHNGMEy_ZBBMe0GYBuBGXBIoLkue2IHcmxF0RmCuh43Tbtdlu4gI5C.I8JJvfsWfLoP5Ss4uj1oiEOegryFluD71vjSB9hHGuYQuJiBPwRjyMTeGPtG2SyPQFwtwg4RN2i04BCz9isz3e1bUofsKsN434COP4mmgqnsyjPLbU-&partNumber=1&A


100%|██████████| 46.9M/46.9M [00:03<00:00, 15.8MB/s]

2025-03-21 20:29:33,390 - root - INFO - Successfully uploaded asset with response status: 200


Scheduled compile job (jp3nd6lz5) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jp3nd6lz5/

Uploading Q2.aimet.zip
2025-03-21 20:29:36,857 - root - INFO - Uploading asset to https://tetrahub-qprod-userdata.s3-accelerate.amazonaws.com/models/mqed2kvym_x5Dr4rz2JzJ3Hjiq.aimet.zip?uploadId=VAGiC.tI6vTQs1kqjwGvjeCaJlvNN4m5Z4IIpEnr6ESNqY47YOfwMyY69l7VkXElVl2gqRTeqec4Teoswuisq796BeURd1MYI8KXPhm.64cu9vTnENYeSq9KtN7OVb0eZqrABpKzJDSmsf7okfjBxSsbirzwz8NNXWdyFZx14WU-&partNumber=1&A


100%|██████████| 46.9M/46.9M [00:02<00:00, 17.5MB/s]

2025-03-21 20:29:39,673 - root - INFO - Successfully uploaded asset with response status: 200


Scheduled compile job (jgozx87dp) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jgozx87dp/



In [12]:
profile_job_tflite = qai_hub.submit_profile_job(

    model = compile_job.get_target_model(),
    device=qai_hub.Device("Snapdragon 8 Elite QRD"),
    name = name + 'tflite'
)
# assert isinstance(compile_job, qai_hub.CompileJob)

# Compile to a QNN Model Library
profile_job_qnn = qai_hub.submit_profile_job(
    model= compile_job_qnn.get_target_model(),
    device=qai_hub.Device("Snapdragon 8 Elite QRD"),
    name = name + 'qnn'
)
# assert isinstance(compile_job_qnn, qai_hub.CompileJob)

Scheduled profile job (jg90ry8vg) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jg90ry8vg/

Scheduled profile job (j572mlkrp) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/j572mlkrp/

